# Workshop_05

## Immagini di esempio:

Per l'arredamento di un'aula sono stati utilizzati i seguenti elementi:
* Cattedra
* Sedia
* Banco
* Appendi Abiti
* Lavagna a muro

<img src = './esempio1.PNG'>

## Codice:

Sono stati creati in primo luogo tutti gli elementi di arredo dell'aula (la creazione di ogni elemento è parametrica rispetto ad una box di contenimento x,y,z).
Solo in seguito gli elementi sono stati disposti all'interno della box di contenimento dell'aula tramite opportune traslazioni e rotazioni.

In [4]:
from larlib import * 


"""ggpl_my_class_furniture(x,y,z) è una funzione che date le dimensioni di una box x, y, z
costruisce al suo interno una classe completa di banchi, sedie, cattedra, lavagna e appendiabiti

@x = x box
@y = y box
@z = z box

"""
def ggpl_my_class_furniture(x,y,z):
    
    
    #FUNZIONI AUSILIARIE ------------------------------------------------------
    """get_chair(x,y,z) è una funzione che date le dimensioni di una box x, y, z
    costruisce al suo interno una sedia
    
    @x = x box
    @y = y box
    @z = z box
    
    """
    def get_chair(x,y,z):

        tLeg = .04 #thickness leg
        #shearing tensor
        HL1 = MAT([[1,0,0,0],[0,1,0.1,0],[0,0,1,0],[0,0,0,1]])
        HL2 = MAT([[1,0,0,0],[0,1,-0.1,0],[0,0,1,0],[0,0,0,1]])

        leg = STRUCT([(CUBOID([tLeg,tLeg,z/2])),
                         T([3])([z/2]),(CUBOID([tLeg,y-tLeg,tLeg])),
                         T([2,3])([y-tLeg,-z/2]),(CUBOID([tLeg,tLeg,z]))])
        legs = STRUCT([HL1(leg),T([1])(x-tLeg),HL2(leg)])

        tSeat = .05 #thickness seat

        fullSeat = MKPOL([[[0,0,0],[x,0,0],[x-.1+tLeg,y,0],[.1-tLeg,y,0]],[[1,2,3,4]],[1]])
        diffSeat = MKPOL([[[0,y,0],[0,y-tLeg,0],[x,y-tLeg,0],[x,y,0]],[[1,2,3,4]],[1]])
        fullSeat = STRUCT([OFFSET([0,0,tSeat]),fullSeat])
        diffSeat = STRUCT([OFFSET([0,0,tSeat]),diffSeat])
        seat = DIFF([fullSeat,diffSeat])

        hseatBack = .2 #height seatback
        seatBack = CUBOID([x-.1,tSeat,hseatBack])

        chair = STRUCT([COLOR(BLACK)(legs),
                        T(3)(z/2+tLeg),COLOR(GREEN)(seat),
                        T([1,2,3])([-(x-.1)/2+x/2,y-tLeg-tSeat,.2]),COLOR(GREEN)(seatBack)])
        return chair

    """get_blackboard(x,y,z) è una funzione che date le dimensioni di una box x, y, z
    costruisce al suo interno una lavagna
    
    @x = x box
    @y = y box
    @z = z box
    
    """
    def get_blackboard(x,y,z):
        tOutline = y #thickness of the outline of the board
        tSlate = .01 #thickness of the slate

        slate = CUBOID([x,tSlate,z])
        outline = OFFSET([tOutline,tOutline,tOutline])(SKEL_1(CUBOID([x-tOutline,tSlate,z-tOutline])))

        blackboard = STRUCT([outline,T([2])([tOutline+tSlate]),COLOR(BLACK)(slate)])
        return blackboard

    """get_desk(x,y,z) è una funzione che date le dimensioni di una box x, y, z
    costruisce al suo interno un banco per alunni
    
    @x = x box
    @y = y box
    @z = z box
    
    """
    def get_desk(x,y,z):
    
        #dimensione fissa per spessore del piano di appoggio di 3 cm
        hPlane = .03 

        #piano d'appoggio con spazio per penne
        plane = STRUCT([COLOR(GREEN)(CUBOID([x,y-.1,hPlane])),
                        T(2)(y-.1),COLOR(BLACK)(CUBOID([x,.09,hPlane-.01])),
                        T(2)(.09),COLOR(GREEN)(CUBOID([x,.01,hPlane]))])

        #gambe tavolo con dim. fissa per raggio gambe tavolo = .02
        rLegs = .02

        leg = CYLINDER([rLegs,z-hPlane])(36)
        legs = STRUCT([T([1,2])([.1,.1]),leg,T([1])([x-.2]),leg,T(2)(y-.2),leg,T(1)(-x+.2),leg]) #posiziono legs

        #dim. fiss per sottobanco con spessori = 2cm , e altezza bordi = 10 cm
        tUndertable = .02
        hUndertable = .1
        undertable = STRUCT([T([1,2])([.1-.02,.1]),CUBOID([tUndertable,y-.2-tUndertable,hUndertable]),
                             T(1)(tUndertable),CUBOID([x-.2,y-.2,tUndertable]),
                             T(1)(x-.2),CUBOID([tUndertable,y-.2-tUndertable,hUndertable]),
                             T([1,2])([.2-x-tUndertable,y-.2-tUndertable]),CUBOID([x-.2+2*tUndertable,tUndertable,hUndertable])])
        undertable = STRUCT([DIFF([undertable,legs])])

        desk = STRUCT([COLOR(BLACK)(legs),T(3)(3*z/4),COLOR(RED)(undertable),T(3)((z/4)-hPlane),plane])

        return desk

    """get_teacher_desk(x,y,z) è una funzione che date le dimensioni di una box x, y, z
    costruisce al suo interno una cattedra
    
    @x = x box
    @y = y box
    @z = z box
    
    """
    def get_teacher_desk(x,y,z):
        #dimensione fissa per spessore del piano di appoggio di 3 cm
        hPlane = .03 

        #piano d'appoggio
        plane = STRUCT([COLOR(BLUE)(CUBOID([x,y,hPlane]))])

        #gambe tavolo con dim. fissa per raggio gambe tavolodi 4 cm
        rLegs = .04

        leg = CYLINDER([rLegs,z-hPlane])(36)
        legs = STRUCT([T([1,2])([.1,.1]),leg,T([1])([x-.2]),leg,T(2)(y-.2),leg,T(1)(-x+.2),leg]) #posiziono legs
        ###### legs = STRUCT([DIFF([legs,plane])])

        #dim. fiss per sottobanco con spessori = 4cm , e altezza bordi = 30 cm
        tCovertable = .04
        hCovertable = y-.3
        covertable = STRUCT([T([1,2])([.1-rLegs/2,.1]),CUBOID([tCovertable,y-.2-tCovertable,hCovertable]),
                             T([1])([x-.2]),CUBOID([tCovertable,y-.2-tCovertable,hCovertable]),
                             T([1,2])([.2-x,y-.2-tCovertable]),CUBOID([x-.2+tCovertable,tCovertable,hCovertable])])
        covertable = STRUCT([DIFF([covertable,legs])])

        desk = STRUCT([COLOR(BLACK)(legs),T(3)((z)-hPlane),plane,T(3)(-hCovertable-.1),COLOR(YELLOW)(covertable)])

        return desk

    """get_wardrobe(x,y,z) è una funzione che date le dimensioni di una box x, y, z
    costruisce al suo interno un appendiabiti da muro
    
    @x = x box
    @y = y box
    @z = z box
    
    """
    def get_wardrobe4(x,y,z):
        structure = CUBOID([x,y/4,z])
        hanger = R([2,3])(PI/2)(CYLINDER([.02,y])(36))
        hanger = T([1,2,3])([x/8,y,z/2])(hanger)
        hangers = STRUCT(NN(4)([hanger,T([1])([x/4])]))
        structure = STRUCT([DIFF([structure,hangers])])
        wardrobe = STRUCT([hangers,COLOR(RED)(structure)])
    
        return wardrobe
    
    #FINE FUNZIONI AUSILIARIE -------------------------------------------------
    
    #alcune variabili predefinite -----------
    doDesk = .5 #distanza orizzontale fra colonne banchi
    dvDesk = 1.5 #distanza verticale fra righe di banchi
    dTeacher = 1.5 #distanza fra banchi e cattedra
    dWardrobe = .05 #distanza fra guardaroba
    dBlackBoard = .8 #distanza fra lavagna e cattedra
    
    #grandezze minime dell'aula per ottenere una matrice 3x4 di banchi
    xMin = 10.
    yMin = 10.
    zMin = 3.
    
    if(x<xMin or y<yMin or z<zMin):
        return None
    
    desk = get_desk(1.5,.6,.8)
    chair = get_chair(.6,.6,.9)
    blackboard = get_blackboard(4.,.05,1.)
    teacherDesk = get_teacher_desk(1.8,.8,.8)
    wardrobe = get_wardrobe4(.6,.05,.2)
    
    #ruoto e sposto lavagna al centro
    blackboard = STRUCT([T([2,3])([y/2-4./2,.8]),R([1,2])(PI/2),blackboard])
    
    teacherDesk = STRUCT([T([1,2])([1.8,.8]),R([1,2])(PI),teacherDesk]) #ruoto cattedra
    teacherDeskWithChair = STRUCT([teacherDesk, T([1,2])([1.8/2-.6/2,.8/2]), chair]) #metto sedia sotto cattedra
    #ruoto e sposto cattedra con sedia al centro
    teacherDeskWithChair = STRUCT([T([1,2])([1.+dBlackBoard,y/2-1.8/2]),R([1,2])(PI/2),teacherDeskWithChair])
    
    desk = STRUCT([T([1,2])([1.5,.6]),R([1,2])(PI),desk])
    deskWithChair = STRUCT([desk, T([1,2])([1.5/2-.6,.6/2]), chair, T([1])([.6]), chair]) #metto sedie
    deskWithChair = STRUCT([T(2)(1.5),R([1,2])(PI+PI/2),(deskWithChair)])
    rowDeskWithChair = STRUCT(NN(3)([deskWithChair,T(2)(1.5+doDesk)])) #costruisco una fila di banchi
    matrixAlumnDesks = STRUCT(NN(4)([rowDeskWithChair,T(1)(dvDesk)])) #costruisco una matrice di banchi 3*4
    matrixAlumnDesks = T([1,2])([dBlackBoard + dTeacher + 1., -5.5/2+y/2])(matrixAlumnDesks) #posizionamento

    wardrobe = R([1,2])(PI/2)(wardrobe)
    wardrobe = STRUCT(NN(5)([wardrobe, T(2)(dWardrobe + .6)]))
    wardrobe = T([1,2,3])([x,-3.2/2+y/2,1.8])(wardrobe)

    aula = STRUCT([blackboard,teacherDeskWithChair,matrixAlumnDesks,wardrobe])
    return aula


### Esempio:

In [ ]:
VIEW(ggpl_my_class_furniture(10.,10.,4.))

#### Singoli elementi
* Sedia

<img src = './chair1.PNG'>

* Banco Alunno (per due alunni)

<img src = './desk1.PNG'>

* Cattedra Insegnante

<img src = './tdesk1.PNG'>

* Guardaroba

<img src = './war1.PNG'>

#### Aula:

<img src = './aula1.PNG'>

<img src = './aula2.PNG'>